In [38]:
import os
import re
import numpy as np
from collections import defaultdict
from math import log


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# Preprocessing function
def preprocess(text):
    return re.findall(r'\b\w+\b', text.lower())


In [41]:

# Load documents
def load_documents(folder_path):
    docs = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r') as file:
                docs[filename] = preprocess(file.read())
    return docs


In [42]:
# Load queries
def load_queries(query_file_path):
    with open(query_file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]


In [43]:
# Compute term frequencies and document frequencies
def compute_statistics(docs):
    doc_count = len(docs)
    term_doc_freq = defaultdict(int)
    term_freq = defaultdict(lambda: defaultdict(int))

    for doc_id, words in docs.items():
        word_set = set(words)
        for word in words:
            term_freq[doc_id][word] += 1
        for word in word_set:
            term_doc_freq[word] += 1

    return term_freq, term_doc_freq, doc_count

In [44]:
# Compute relevance probabilities using BIM
def compute_relevance_prob(query, term_freq, term_doc_freq, doc_count):
    scores = {}
    for doc_id in term_freq:
        score = 1.0
        for term in query:
            tf = term_freq[doc_id].get(term, 0)
            df = term_doc_freq.get(term, 0)
            p_term_given_relevant = (tf + 1) / (sum(term_freq[doc_id].values()) + len(term_doc_freq))
            p_term_given_not_relevant = (df + 1) / (doc_count - df + len(term_doc_freq))
            score *= (p_term_given_relevant / p_term_given_not_relevant)
        scores[doc_id] = score
    return scores

In [46]:
import os

# Main retrieval function
def retrieve_documents(folder_path, query_file_path):
    docs = load_documents(folder_path)
    queries = load_queries(query_file_path)

    term_freq, term_doc_freq, doc_count = compute_statistics(docs)

    # Define the output file path in the same directory as the dataset folder
    output_file_path = os.path.join(folder_path, 'Abin_result.txt')

    # Open a file to write results in the same directory as the dataset
    with open(output_file_path, 'w') as result_file:
        for query in queries:
            query_terms = preprocess(query)
            scores = compute_relevance_prob(query_terms, term_freq, term_doc_freq, doc_count)
            ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)

            # Print to console and write to file
            print(f"Query: {query}")
            result_file.write(f"Query: {query}\n")

            # Print and write only the top 3 results with ranks
            for rank, (doc_id, score) in enumerate(ranked_docs[:3], start=1):
                print(f"Rank {rank}: Document {doc_id}, Score: {score:.4f}")
                result_file.write(f"Rank {rank}: Document {doc_id}, Score: {score:.4f}\n")

            print()
            result_file.write("\n")


folder_path = '/content/drive/MyDrive/info_retrieval_lab/Class_wk4'
query_file_path = '/content/drive/MyDrive/info_retrieval_lab/Final project/queries2.txt'
retrieve_documents(folder_path, query_file_path)


Query: ﻿to find the perfect airline that is
Rank 1: Document Qatar Airway.txt, Score: 0.0008
Rank 2: Document British Airways.txt, Score: 0.0005
Rank 3: Document Cathay pacific.txt, Score: 0.0005

Query: known for its comfort and reliability
Rank 1: Document Delta Airline.txt, Score: 0.0031
Rank 2: Document Air france.txt, Score: 0.0014
Rank 3: Document Lufthansa.txt, Score: 0.0014

Query: suitable for budget-conscious travelers
Rank 1: Document ANA(All Nippon Airways).txt, Score: 0.0299
Rank 2: Document Lufthansa.txt, Score: 0.0298
Rank 3: Document Emirates.txt, Score: 0.0272

Query: airlines that are known for their premium service
Rank 1: Document Singapore Airline.txt, Score: 0.0000
Rank 2: Document American Airline.txt, Score: 0.0000
Rank 3: Document Emirates.txt, Score: 0.0000

Query: airlines with hypoallergenic meals and special diet options
Rank 1: Document American Airline.txt, Score: 0.0017
Rank 2: Document Singapore Airline.txt, Score: 0.0007
Rank 3: Document Qantas.txt, Sc